---
title: Step 03 - Featurization, Vectorization, and Pre-Modeling
subject: Churn Analysis
subtitle: Step 03 - Featurization, Vectorization, and Pre-Modeling - Churn Analysis
short_title: Featurization, Vectorization, and Pre-Modeling
authors:
  - name: Jane Doe
    affiliations:
      - University of California, Berkeley
      - STAT 159/259: Reproducible and Collaborative Statistical Data Science 
    orcid: 0000-0000-0000-0000
    email: jdoe@berkeley.edu
license: TBD
keywords: featurization, vectorization, pre-modeling, churn, spotify
---

# Step 03: Featurization, Vectorization, and Pre-Modeling